In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
from torch.autograd import Variable

#log_pX = - beta1 * abs(X) - beta2 * X * X

def mcmc_simulate_beta(Y_0 ,langevin_step_size ,langevin_step_num, image_size, beta1, beta2):
    #Y_tau = Y_0
    Y_tau = Y_0.clone().requires_grad_(True)
    for i in range(langevin_step_num):
        if Y_tau.grad is not None:
            Y_tau.grad.data.zero_()
        #Y_tau = Variable(Y_tau, requires_grad=True)
        noise = torch.randn(image_size)
        log_pY = (- beta1 * abs(Y_tau) - beta2 * Y_tau ** 2).sum()
        log_pY.backward()
        #print(i, Y_tau, noise, Y_tau.grad)
        #Y_tau = Y_tau + 0.5 * langevin_step_size * langevin_step_size * \
        #(Y_tau.grad) + langevin_step_size * noise
        Y_diff = 0.5 * (langevin_step_size**2)*Y_tau.grad + langevin_step_size * noise
        #print(i, Y_tau.grad.norm()) # Y_tau, noise, Y_tau.grad)
        Y_tau.data.add_(Y_diff.data)
        #print(Y_tau)
    return Y_tau.data.numpy()

langevin_step_size = 0.001  
langevin_step_num = 1000
image_size = 64
beta1 = 5
beta2 = 0
Y_0 = torch.randn(image_size)
Y_u0 = torch.rand(image_size)
Y_u = torch.ones(image_size)


Y_o = mcmc_simulate_beta(Y_u ,langevin_step_size ,langevin_step_num, image_size, beta1, beta2)
print(Y_o)

[1.0056665  0.95453507 0.9830859  1.0180733  1.0551648  0.9137835
 0.98802835 0.96082133 1.0201755  1.0193856  0.9486091  1.0544868
 0.97877645 0.9986358  0.9971437  0.99543065 1.0634589  1.0084629
 0.95674086 0.97850263 0.9871521  0.9992561  0.9923762  1.0032971
 0.9928005  0.9659137  1.006833   1.0250133  0.9983864  0.98697346
 1.0431799  1.0241141  1.003543   0.99690354 0.9570714  1.0327413
 1.0244141  1.0223829  0.9750982  0.9397878  1.0142442  0.99315244
 1.035537   1.0121608  0.98292285 1.0432117  0.99114066 1.0241135
 0.99573594 0.98896086 0.99485976 1.0111951  0.99738544 0.9942554
 0.9835643  0.9809945  0.97753006 0.97933584 0.9853418  0.9941051
 0.99225754 1.0052447  1.0135239  1.0207292 ]


In [2]:
sigma = 1
Y_data_gauss = sigma * torch.randn(image_size).numpy()
print(Y_data_gauss)

def mle_simulate_beta(Y_data, Y_tuta0, beta1_0, beta2_0, mle_step_num, mle_step_size,langevin_step_size, langevin_step_num, image_size):
    beta1 = beta1_0
    beta2 = beta2_0
    Y_tuta = Y_tuta0
    for i in range(mle_step_num):
        Y_tuta = mcmc_simulate_beta(torch.Tensor(Y_tuta), langevin_step_size, langevin_step_num, image_size, beta1, beta2)
        #Y_tuta = mcmc_simulate_beta(Y_tuta0, langevin_step_size, langevin_step_num, image_size, beta1, beta2)
        d_beta1 = ((-abs(Y_data).sum()) - (-abs(Y_tuta).sum())) / image_size
        d_beta2 = ((-(Y_data ** 2).sum()) - (-(Y_tuta ** 2).sum())) / image_size
        beta1 = beta1 - mle_step_size * d_beta1
        beta2 = beta2 - mle_step_size * d_beta2
        print(beta1,beta2)
        #print(i,Y_tuta)
    return beta1,beta2

import pylab as plt
import matplotlib.mlab as mlab
plt.figure(figsize=(10,10))
plt.subplot(311)
plt.psd(Y_o, NFFT=150, Fs=100000, window=mlab.window_none, pad_to=512, noverlap=75,
        scale_by_freq=True)

plt.figure(figsize=(10,10))
plt.subplot(312)
plt.psd(Y_data_gauss, NFFT=150, Fs=100000, window=mlab.window_none, pad_to=512, noverlap=75,
        scale_by_freq=True)

[ 0.01445745  1.3891363   0.17198949 -0.19919778  1.2418554   1.5216422
 -1.1963961   1.4695461   0.03147738 -0.90033     0.36926135 -0.6107449
 -1.2056278   0.29430488 -0.53748065 -0.47834444  0.96351904  0.17959099
  0.6092097  -0.19770314  0.14680803  0.60875154  1.1038625  -0.155281
 -1.8180841  -0.5322398   0.5647909  -1.5601048   0.14249042 -1.2877266
  1.2101339  -0.08299363 -0.9736558   0.08254275  0.5036063   0.8246629
 -0.04128933  1.3378147  -1.1027924   0.18723267  0.45808697 -0.58130264
  0.04511406  0.770851    0.35109058  0.6834546  -0.52604043  0.74421144
  0.2812765  -1.2447708   0.04975095  1.0197822  -0.8801633   0.19098918
  0.3423406   1.5318979  -0.23127516 -0.8061243  -0.8873862  -0.84532106
  2.2191067   0.7142274  -0.7281015  -0.37627345]


(array([1.28136480e-06, 2.31717399e-06, 1.67210669e-06, 8.71862596e-07,
        2.32100163e-07, 3.41669448e-08, 4.30183633e-07, 1.39051713e-06,
        2.71115664e-06, 4.07796961e-06, 5.16492446e-06, 5.73152738e-06,
        5.68590076e-06, 5.09388292e-06, 4.13611951e-06, 3.03527258e-06,
        1.98561520e-06, 1.11294855e-06, 4.76066845e-07, 9.99232412e-08,
        1.54055133e-08, 2.78858920e-07, 9.57042062e-07, 2.08377423e-06,
        3.61235470e-06, 5.39326493e-06, 7.19612171e-06, 8.77311231e-06,
        9.93897811e-06, 1.06317239e-05, 1.09247931e-05, 1.09830835e-05,
        1.09815455e-05, 1.10231376e-05, 1.10935180e-05, 1.10724541e-05,
        1.07950809e-05, 1.01329915e-05, 9.05711659e-06, 7.65485674e-06,
        6.09710060e-06, 4.57469196e-06, 3.23625522e-06, 2.15515024e-06,
        1.33530807e-06, 7.44823581e-07, 3.53328971e-07, 1.50897975e-07,
        1.40362133e-07, 3.12624309e-07, 6.25251857e-07, 1.00221143e-06,
        1.35879418e-06, 1.63908680e-06, 1.84395515e-06, 2.031438

In [3]:
Y_tuta0 = torch.randn(image_size)
beta1_0 = 0
beta2_0 = 0.9
mle_step_num = 1000
mle_step_size = 0.01
langevin_step_num = 1000
#beta1_o,beta2_o = mle_simulate_beta(Y_o, Y_tuta0, beta1_0, beta2_0, mle_step_num, mle_step_size,langevin_step_size, langevin_step_num, image_size)
beta1_o,beta2_o = mle_simulate_beta(Y_data_gauss, Y_tuta0, beta1_0, beta2_0, mle_step_num, mle_step_size,langevin_step_size, langevin_step_num, image_size)
print(beta1_o,beta2_o)

-0.0006925398111343383 0.8976246547698975
-0.0013389605283737182 0.8952697902917862
-0.0019878798723220823 0.8929322355985642
-0.002588696479797363 0.8906555432081222
-0.0032171344757080077 0.888275573849678
-0.003917630314826965 0.8857686930894851
-0.00463869035243988 0.8832898563146591
-0.005365000367164612 0.88085766851902
-0.006085147261619568 0.8784382706880569
-0.00679194986820221 0.8759897422790527
-0.007508088350296021 0.8735336935520172
-0.008149266242980957 0.8711524283885955
-0.00865398108959198 0.8689843118190764
-0.009150147438049316 0.8667485958337783
-0.00961318016052246 0.8645849305391311
-0.010056292414665222 0.8624309509992599
-0.010523331761360168 0.8602872002124786
-0.011009123921394347 0.8581445872783661
-0.01151581287384033 0.8560053324699402
-0.011944511532783506 0.854002873301506
-0.012361267805099486 0.85203562438488
-0.01277196705341339 0.8501297712326049
-0.013269919157028198 0.8480569112300872
-0.013699745535850525 0.8460651302337646
-0.01415236234664917 0.8

-0.06915584444999694 0.5598141396045684
-0.06899058103561401 0.5593791621923445
-0.06884353458881377 0.5588657343387602
-0.06876878261566162 0.558245277404785
-0.06866865158081055 0.5576027715206144
-0.06859132468700409 0.5569134771823882
-0.06854061961174011 0.5561304247379302
-0.06849167764186859 0.555274468064308
-0.06846580862998962 0.5543574720621107
-0.0683740246295929 0.553559300303459
-0.06826133251190186 0.5527463877201079
-0.06813572764396668 0.551988043785095
-0.06802127122879029 0.5512168991565702
-0.06798054397106171 0.5503600662946699
-0.06793889045715332 0.5495565903186796
-0.06783362209796905 0.5488968563079832
-0.06778580844402313 0.548165782690048
-0.0677575033903122 0.547407555580139
-0.06761511206626893 0.5468481194972991
-0.06745593667030335 0.5462361228466033
-0.06728200852870941 0.5456122612953185
-0.06722330331802369 0.5447905242443083
-0.06712231695652009 0.5439910531044004
-0.06711361467838288 0.5431090426445006
-0.06716576278209686 0.5421472221612929
-0.06717

-0.04938908755779271 0.46830541789531677
-0.04928739607334142 0.4682817071676251
-0.04921677291393285 0.46827917635440797
-0.04918359279632573 0.468190874457359
-0.04911149084568028 0.46816515266895264
-0.04902372002601628 0.4682075178623196
-0.04890539228916173 0.4682726907730099
-0.04884336709976201 0.4682559812068936
-0.048768419027328536 0.4682449877262112
-0.048719733357429545 0.4682400268316265
-0.04871251046657566 0.46814784765243495
-0.0486232960224152 0.4682396268844601
-0.04858941078186039 0.46826004385948145
-0.04850705862045292 0.4682960683107372
-0.04838243544101719 0.4684130769968029
-0.0481960773468018 0.46862715542316397
-0.04798103153705601 0.4688336068391796
-0.04777110219001774 0.4690661209821697
-0.047471893429756204 0.46943042576312977
-0.04716786503791813 0.46980521738529163
-0.04685638606548313 0.4702217000722881
-0.04657845258712773 0.4706209582090374
-0.046292344927787825 0.4710002565383907
-0.045933306217193645 0.47147559344768486
-0.045605597496032754 0.47196

0.013899528384208635 0.5641332232952118
0.014467128515243486 0.5651790285110473
0.01503737866878505 0.5662435382604598
0.015576546788215592 0.5672560709714889
0.01605425298213954 0.5681543385982513
0.016591586470603898 0.5691276395320891
0.017069893479347183 0.5700030487775801
0.01754769623279567 0.5708798074722289
0.01804044604301448 0.5718078607320785
0.018547412157058667 0.5727215564250945
0.018977355360984754 0.5735749065876006
0.019361749887466383 0.5743622279167174
0.019745033383369397 0.575128309726715
0.020124263167381237 0.5758707273006438
0.020525386333465526 0.5766685950756072
0.020913575887680003 0.5774524211883544
0.02132361829280848 0.5782740110158919
0.021730214953422495 0.5791104102134703
0.022152665257453866 0.5799367511272429
0.02253744244575495 0.5808135426044462
0.02288155138492579 0.5816651135683057
0.02323825716972346 0.5825463193655012
0.023567155599594065 0.5834009420871732
0.023834865689277597 0.5841459596157071
0.02410136044025416 0.5849228149652479
0.02435097

0.05458257257938382 0.7194056850671761
0.05476853191852567 0.7204686212539665
0.05498385548591611 0.7215442818403236
0.0551963937282562 0.7226017838716499
0.05548764646053312 0.7237503927946083
0.05573837876319883 0.7248646020889274
0.05603924453258512 0.7260227513313285
0.0563376975059509 0.7271595293283455
0.05662334442138669 0.7282139444351189
0.056950729489326446 0.7293387687206261
0.057308025956153835 0.7305034011602394
0.057690653800964324 0.7317104882001869
0.05810010492801663 0.732992464900016
0.05843256950378414 0.734113520383834
0.058713261485099755 0.7351271635293952
0.0590001559257507 0.7360896211862555
0.059262301325797996 0.7370103007554999
0.05955181181430813 0.7379424244165411
0.05985973954200741 0.7389251095056525
0.0601333528757095 0.7397787046432486
0.0604449367523193 0.7407093942165366
0.06078740179538723 0.7417419409751883
0.061070314645767176 0.742715768814086
0.06132885098457333 0.7437155348062505
0.06155464708805081 0.7446608597040166
0.061824128627777065 0.7456

In [4]:
Y_tuta0 = torch.randn(image_size)
beta1_0 = 4.9
beta2_0 = 0
mle_step_num = 1000
mle_step_size = 0.01
langevin_step_num = 1000
beta1_o,beta2_o = mle_simulate_beta(Y_o, Y_tuta0, beta1_0, beta2_0, mle_step_num, mle_step_size,langevin_step_size, langevin_step_num, image_size)
#beta1_o,beta2_o = mle_simulate_beta(Y_data_gauss, Y_tuta0, beta1_0, beta2_0, mle_step_num, mle_step_size,langevin_step_size, langevin_step_num, image_size)
print(beta1_o,beta2_o)

4.901554816961289 -0.001389405131340027
4.90311477303505 -0.0027610981464385987
4.904690429568291 -0.004113863110542297
4.906267014145851 -0.005403498411178589
4.907875555157662 -0.006533008217811585
4.9094356483221055 -0.00775944471359253
4.910945647358894 -0.009102508425712585
4.912501860260964 -0.010315837860107422
4.914087520241737 -0.011438454985618591
4.9156878858804705 -0.012603517770767212
4.917276692390442 -0.013692513108253479
4.918882563114167 -0.014689598083496094
4.920496722459793 -0.015672701001167296
4.922083376049995 -0.01672804832458496
4.923715616464615 -0.01778785765171051
4.92536124587059 -0.018860884904861448
4.926991091966629 -0.019869536757469174
4.928583714962006 -0.020858645439147946
4.930228089094162 -0.02175855934619903
4.932005168199539 -0.02251973748207092
4.933759736418724 -0.02335915625095367
4.935521898269653 -0.024081819057464596
4.937263758182525 -0.02489255368709564
4.938904658555984 -0.02593427181243896
4.940522019863128 -0.027056125998497008
4.94220

5.559881122112269 0.34616134554147715
5.564562193155283 0.35120000511407845
5.569290586113924 0.3562802603840827
5.574054905772203 0.3613625100255012
5.578777051568025 0.36648717910051337
5.583445393443101 0.3715489783883094
5.588200927376741 0.3766884317994117
5.592941282391542 0.3817979958653449
5.5977002555131845 0.38693725556135167
5.602429513335221 0.39202233403921116
5.607176694273942 0.3971649959683417
5.611960382461541 0.40232494860887513
5.616742599010461 0.4075465884804724
5.621522997617715 0.4127845939993857
5.626269713640207 0.41797283142805086
5.631090927124018 0.42322964817285524
5.635929551124567 0.4285076269507407
5.640796043276781 0.4338119003176688
5.645707169175142 0.4392018303275107
5.650620965361589 0.44458531111478794
5.655501371622079 0.44988791316747656
5.660451327562326 0.4553208437561988
5.6654712855815825 0.46079445689916604
5.670486733913416 0.46621629983186713
5.675494203567499 0.47165432780981054
5.680495582818979 0.4770548877120017
5.68550093531608 0.4823

6.957912751138195 1.9337510791420929
6.965629620254024 1.9424100187420836
6.973320680558666 1.9510780093073836
6.981013072431072 1.9597074648737898
6.988698825538143 1.968340326845645
6.996404028832897 1.9770042327046384
7.004121862947925 1.9857009932398786
7.011847099363788 1.9943978264927853
7.019638382494434 2.003141746819018
7.027452339231952 2.011899805366992
7.035247481167301 2.020657607614993
7.043052686154827 2.029450775682925
7.050895495712742 2.0382559499144546
7.058755612075314 2.047107686698436
7.066648112237439 2.0559884282946577
7.074498535692677 2.0648505279421796
7.082341161668286 2.0737033101916302
7.090228379666791 2.0825986060500132
7.098109087645994 2.091490075290202
7.106011116206632 2.100383720099925
7.113894409835325 2.1092447838187205
7.121755314171301 2.1180996164679513
7.129609668552862 2.126960507333277
7.137438305318343 2.1358233287930473
7.145253063738333 2.1446355226635916
7.153083896338926 2.153453995883463
7.160964785516248 2.1622924861311894
7.168871037

8.82939511448143 4.007871566712854
8.838355441987495 4.017544656097887
8.847342714965324 4.027234046757219
8.856319513618926 4.036926053464411
8.865316707789878 4.046629678905008
8.874356409013252 4.056332171857354
8.883372697532158 4.066019362509248
8.892353368699531 4.075710063874719
8.901410749256591 4.0854279026389095
8.910451532304267 4.0951467391848535
8.919474992454031 4.10486953526735
8.92855019301174 4.114617556035516
8.93771268159149 4.124378782212732
8.946869488060454 4.134155627191064
8.95602049022911 4.143932423889635
8.965195332467536 4.153713734447954
8.974339012801627 4.163491620123384
8.983505531847456 4.173283617198465
8.99262097865341 4.18305857926607
9.001756475269772 4.192840084731578
9.010897220671154 4.202625006139277
9.020032836496807 4.212408209741114
9.029122026264645 4.222169681489466
9.038267324268794 4.231942155659197
9.047402808368183 4.2417187693715075
9.056512413322903 4.251488640010355
9.065657917559124 4.2612793371081334
9.074816292822337 4.27105531781

10.906549328863603 6.239669538438317
10.915758698880655 6.2495315799117055
10.924974819123728 6.259392816722389
10.93417191654442 6.269252597987649
10.943348723351939 6.279104814827439
10.952551412284357 6.28896208554506
10.96176231235264 6.298827409446236
10.970937866270525 6.30868425518274
10.980150095522387 6.318545351922509
10.989313451349718 6.3283967247605295
10.99847553521393 6.338248538076875
11.007641493976099 6.348103350102899
11.016851173937303 6.357967839539048
11.026071225702745 6.367836303412911
11.035284664928897 6.377701967656609
11.044473393261416 6.387563586533067
11.05366072148083 6.397428883612153
11.062829978764041 6.4072899672389
11.071978662312015 6.417136488854882
11.081144483387455 6.426981200277802
11.090291787087901 6.436823328435418
11.099492646753772 6.446669425666329
11.108692058622822 6.456512778103348
11.117880392372593 6.466359430253502
11.127063861787304 6.476208736598488
11.136274926960454 6.486063188612457
11.145487017333492 6.4959232202172235
11.154